In [1]:
from flask import Flask, request, render_template
import joblib
import pandas as pd
import numpy as np
import flask

In [2]:
# Load your trained machine learning model
model = joblib.load(open('gradient_boosting_regressor.pkl','rb'))

app = flask.Flask(__name__, template_folder='templates', static_folder='static')

@app.route('/')
def main():
    return(flask.render_template('totalpurchases.html'))

@app.route('/predict', methods=['POST'])
def predict():
    
   # Parsing the input data from the request and appending it to the DataFrame
    active   = request.form['active']
    frequency = request.form['frequency']
    discountsused = request.form['discountsused']
    totalrevenue = request.form['totalrevenue']
    feedbackscore = request.form['feedbackscore']
    preferredpaymentmethod = request.form['preferredpaymentmethod']
    firstpurchasedate = request.form['firstpurchasedate']
    lastpurchasedate = request.form['lastpurchasedate']
    
    df = pd.DataFrame({'totalrevenue': [totalrevenue], 'churnindicator': [active], 'discountsused': [discountsused], 'feedbackscore': [feedbackscore], 'preferredpaymentmethod': [preferredpaymentmethod], 
                       'frequency': [frequency], 'firstpurchasedate': [firstpurchasedate], 'lastpurchasedate': [lastpurchasedate]})

    mapping = {'yes': 1, 'no': 0}
    mapping1 = {'debit card': 1, 'cash': 2, 'credit card': 4, 'afterpay': 5, 'google pay': 3, 'apple pay': 3, 'sezzle': 3, 'paypal': 3, 'eft': 3}
    
    # Applying the mapping 
    df['churnindicator'] = df['churnindicator'].map(mapping)
    df['preferredpaymentmethod'] = df['preferredpaymentmethod'].map(mapping1)
    
    # converting 'lastpurchasedate' and 'firstpurchasedate' into datetime datatype
    df['lastpurchasedate'] = pd.to_datetime(df['lastpurchasedate'])
    df['firstpurchasedate'] = pd.to_datetime(df['firstpurchasedate'])

    # getting the max from 'lastpurchasedate'
    current_date = df['lastpurchasedate'].max()
    # calculating 'recency' based on 'lastpurchasedate'
    df['recency'] = (current_date - df['lastpurchasedate']).dt.days
    
    # creatiing 'Tenure' column by subracting the 'firstpurchasedate' from 'lastpurchasedate' for each customer
    df['tenure'] = (df['lastpurchasedate'] - df['firstpurchasedate']).dt.days

    features = ['totalrevenue', 'churnindicator', 'discountsused', 'feedbackscore', 'preferredpaymentmethod', 'frequency', 'tenure', 'recency']
    
    # Create a new DataFrame with only the selected columns
    data = df[features]

    # Making predictions using the loaded model and the data stored in the DataFrame
    prediction = model.predict(data)  
    
    # Round the predicted values to the nearest integer
    prediction = prediction.round().astype(int)
    
    #Values to be returned on making predictions
    prediction
    
    # Returning prediction result as plain text
    return flask.render_template('totalpurchases.html', prediction=prediction)

if __name__ == '__main__':
    app.run(debug=True, port = 5001, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2024 18:39:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 18:39:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 18:39:22] "GET /static/background_2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2024 18:39:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Apr/2024 18:40:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [26/Apr/2024 18:40:09] "GET /static/background_2.jpg HTTP/1.1" 304 -
127.0.0.1 - - [26/Apr/2024 18:40:09] "GET /predict HTTP/1.1" 405 -
